<a href="https://colab.research.google.com/github/DilemmaFixer3/AI_pr_5-6-7/blob/main/pr8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm # Для відображення прогресу

# Встановіть фіксоване початкове значення для відтворюваності
torch.manual_seed(42)
np.random.seed(42)

# --- 0. Завантаження та попередня обробка даних ---

def load_and_preprocess_data(file_path, sample_frac=0.2):
    """
    Оновлена функція: завантажує, зменшує, видаляє рідкісні класи
    та розділяє дані з використанням стратифікації.
    """
    try:
        df = pd.read_csv(file_path)
        # Зменшення датасету до 20%
        df = df.sample(frac=sample_frac, random_state=42).reset_index(drop=True)
    except FileNotFoundError:
        print(f"Помилка: Файл '{file_path}' не знайдено.")
        return None, None, None, None, None

    df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
    disease_col = next((col for col in df.columns if 'disease' in col.lower()), None)
    if not disease_col:
        print("Помилка: Не вдалося знайти стовпець з назвою хвороби (наприклад, 'Disease').")
        return None, None, None, None, None

    X = df.drop(columns=[disease_col])
    y_labels = df[disease_col]

    # --- ВИПРАВЛЕННЯ ПОМИЛКИ: Видалення класів з одним зразком ---
    class_counts = y_labels.value_counts()
    rare_classes = class_counts[class_counts < 2].index.tolist()

    if rare_classes:
        print(f" Видаляємо {len(rare_classes)} рідкісних класів (хвороб), що мають < 2 зразків.")
        df_filtered = df[~df[disease_col].isin(rare_classes)]

        X = df_filtered.drop(columns=[disease_col])
        y_labels = df_filtered[disease_col]

        if X.empty:
            print("Помилка: Після фільтрації не залишилося даних.")
            return None, None, None, None, None

    # Перетворення міток (хвороб) на числові значення
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y_labels)

    # Перетворення на тензори
    X_tensor = torch.tensor(X.values, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)

    # Розділення на тренувальний та тестовий набори (стратифікація тепер можлива)
    X_train, X_test, y_train, y_test = train_test_split(
        X_tensor, y_tensor, test_size=0.2, random_state=42, stratify=y_tensor
    )

    print(f" Успішно завантажено. Кількість класів: {len(label_encoder.classes_)}")
    return X_train, X_test, y_train, y_test, len(label_encoder.classes_)

# Вкажіть правильний шлях до файлу
FILE_PATH = 'Final_Augmented_dataset_Diseases_and_Symptoms.csv' # Замініть на ім'я вашого файлу
X_train, X_test, y_train, y_test, NUM_CLASSES = load_and_preprocess_data(FILE_PATH, sample_frac=0.2)

if X_train is None:
    exit()

INPUT_SIZE = X_train.shape[1]
BATCH_SIZE = 64
N_EPOCHS = 20 # Зменшимо кількість епох для швидшого виконання

# Створення об'єктів Dataset та DataLoader
class SymptomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = SymptomDataset(X_train, y_train)
test_dataset = SymptomDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- 1. Визначення моделі нейронної мережі ---

class SimpleClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.layer_stack(x)

# --- 2. Адаптація оптимізаторів: Власна версія Adam ---

class CustomAdam(optim.Optimizer):
    """
    Власна реалізація оптимізатора Adam з регульованими параметрами.
    Наслідує torch.optim.Optimizer.
    """
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        if not 0.0 <= lr:
            raise ValueError(f"Неприпустиме значення швидкості навчання: {lr}")
        if not 0.0 <= eps:
            raise ValueError(f"Неприпустиме значення epsilon: {eps}")
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError(f"Неприпустиме значення beta1: {betas[0]}")
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError(f"Неприпустиме значення beta2: {betas[1]}")
        if not 0.0 <= weight_decay:
            raise ValueError(f"Неприпустиме значення weight_decay: {weight_decay}")

        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        super().__init__(params, defaults)

    def step(self, closure=None):
        """Виконує єдиний крок оптимізації."""
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('Adam не підтримує розріджені градієнти')

                state = self.state[p]

                # Ініціалізація стану (моментів)
                if not state:
                    state['step'] = 0
                    # Експоненційно-зважене середнє градієнтів
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Експоненційно-зважене середнє квадратів градієнтів
                    state['exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1
                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']

                # 1. Weight decay (L2 регуляризація) - застосовується як у AdamW для кращої продуктивності
                # У класичному Adam його застосовують до градієнта, але тут робимо AdamW-подібний підхід
                if group['weight_decay'] != 0:
                    grad = grad.add(p.data, alpha=group['weight_decay'])

                # 2. Оновлення першого моменту (exp_avg)
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)

                # 3. Оновлення другого моменту (exp_avg_sq)
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                # 4. Корекція зміщення (Bias correction)
                denom = (exp_avg_sq.sqrt() / np.sqrt(bias_correction2)).add_(group['eps'])
                step_size = group['lr'] / bias_correction1

                # 5. Оновлення параметрів
                p.data.addcdiv_(exp_avg, denom, value=-step_size)

        return loss


# --- 3. Функції для тренування та оцінки моделі ---

def train_model(model, optimizer, criterion, train_loader, n_epochs):
    """Тренує модель і повертає точність на тренувальних даних."""
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct_predictions / total_samples
    return avg_loss, accuracy

def evaluate_model(model, test_loader, criterion):
    """Оцінює модель на тестових даних."""
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct_predictions / total_samples
    return avg_loss, accuracy

# --- 4. Експериментальний цикл для виконання завдань ---

def run_experiment(model_class, input_size, num_classes, train_loader, test_loader,
                   optimizer_config, n_epochs):
    """Проводить тренування моделі з заданим оптимізатором і швидкістю навчання."""

    # Ініціалізація моделі та критерію втрат
    model = model_class(input_size, num_classes)
    criterion = nn.CrossEntropyLoss()

    # Ініціалізація оптимізатора
    optim_name = optimizer_config['name']
    lr = optimizer_config['lr']

    if optim_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif optim_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optim_name == 'CustomAdam':
        optimizer = CustomAdam(model.parameters(), lr=lr)
    elif optim_name == 'NAdam':
        optimizer = optim.NAdam(model.parameters(), lr=lr)
    elif optim_name == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=lr)
    else:
        raise ValueError(f"Невідомий оптимізатор: {optim_name}")

    # Тренування
    for epoch in range(n_epochs):
        train_loss, train_acc = train_model(model, optimizer, criterion, train_loader, 1)
        # print(f"Epoch {epoch+1}/{n_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")

    # Оцінка
    test_loss, test_acc = evaluate_model(model, test_loader, criterion)

    return train_acc, test_acc, test_loss


# --- Головний блок виконання завдань ---
def main_lab_work():

    print(f"---  Розмірність даних: {INPUT_SIZE} ознак, {NUM_CLASSES} класів ---")
    print(f"---  Тренування на {N_EPOCHS} епохах ---")

    # 1. Дослідження впливу швидкості навчання (Learning Rate)
    print("\n## 1. Дослідження впливу швидкості навчання 🧪")

    optimizers_to_test = ['SGD', 'Adam']
    learning_rates = [1e-4, 1e-3, 1e-2]

    results_lr = []

    for opt_name in optimizers_to_test:
        for lr in learning_rates:
            print(f"-> Оптимізатор: {opt_name}, Швидкість навчання (LR): {lr}")
            config = {'name': opt_name, 'lr': lr}

            # Виконання експерименту
            train_acc, test_acc, _ = run_experiment(
                SimpleClassifier, INPUT_SIZE, NUM_CLASSES,
                train_loader, test_loader, config, N_EPOCHS
            )

            results_lr.append({
                'Optimizer': opt_name,
                'LR': lr,
                'Train Accuracy': f"{train_acc:.2f}%",
                'Test Accuracy': f"{test_acc:.2f}%"
            })

    print(pd.DataFrame(results_lr).to_markdown(index=False))


    # 2. Адаптація оптимізаторів: Власна версія Adam
    # Використаємо CustomAdam з оптимальним LR, знайденим у п.1 (або стандартним 1e-3)
    print("\n## 2. Адаптація оптимізаторів: Власна версія Adam (CustomAdam) ")

    # Порівняння CustomAdam з вбудованим Adam
    best_lr_adam = 1e-3

    custom_adam_config = {'name': 'CustomAdam', 'lr': best_lr_adam}
    print(f"-> CustomAdam (LR: {best_lr_adam})")
    custom_train_acc, custom_test_acc, _ = run_experiment(
        SimpleClassifier, INPUT_SIZE, NUM_CLASSES,
        train_loader, test_loader, custom_adam_config, N_EPOCHS
    )

    adam_config = {'name': 'Adam', 'lr': best_lr_adam}
    print(f"-> Вбудований Adam (LR: {best_lr_adam})")
    adam_train_acc, adam_test_acc, _ = run_experiment(
        SimpleClassifier, INPUT_SIZE, NUM_CLASSES,
        train_loader, test_loader, adam_config, N_EPOCHS
    )

    results_custom = [{
        'Optimizer': 'CustomAdam',
        'LR': best_lr_adam,
        'Test Accuracy': f"{custom_test_acc:.2f}%"
    }, {
        'Optimizer': 'Built-in Adam',
        'LR': best_lr_adam,
        'Test Accuracy': f"{adam_test_acc:.2f}%"
    }]

    print(pd.DataFrame(results_custom).to_markdown(index=False))


    # 3. Порівняння з новими оптимізаторами (Nadam, AdamW)
    print("\n## 3. Порівняння з новими оптимізаторами (Nadam, AdamW)")

    # Використаємо стандартний LR (1e-3)
    new_optimizers = ['NAdam', 'AdamW']
    default_lr = 1e-3

    results_new_opt = []

    for opt_name in new_optimizers:
        print(f"-> Оптимізатор: {opt_name}, Швидкість навчання (LR): {default_lr}")
        config = {'name': opt_name, 'lr': default_lr}

        train_acc, test_acc, _ = run_experiment(
            SimpleClassifier, INPUT_SIZE, NUM_CLASSES,
            train_loader, test_loader, config, N_EPOCHS
        )

        results_new_opt.append({
            'Optimizer': opt_name,
            'LR': default_lr,
            'Test Accuracy': f"{test_acc:.2f}%"
        })

    # Додамо Adam для порівняння
    results_new_opt.append({
        'Optimizer': 'Built-in Adam',
        'LR': default_lr,
        'Test Accuracy': f"{adam_test_acc:.2f}%"
    })

    print(pd.DataFrame(results_new_opt).to_markdown(index=False))


    # 4. Аналіз на реальних медичних даних
    print("\n## 4. Аналіз на реальних медичних даних ")
    print("> Тренування та оцінка проводилися на 20% реальних медичних даних,")
    print("> як зазначено у вхідних даних (X_train, X_test, y_train, y_test).")
    print(f"> Загальна кількість записів у 20% вибірці: {X_train.shape[0] + X_test.shape[0]}.")

    # Використовуємо найкращий оптимізатор з усіх експериментів для фінальної оцінки (наприклад, AdamW)
    best_opt_config = {'name': 'AdamW', 'lr': 1e-3}
    final_model = SimpleClassifier(INPUT_SIZE, NUM_CLASSES)
    final_optimizer = optim.AdamW(final_model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    print("\n-> Фінальне тренування моделі з AdamW:")

    # Тренування з прогрес-баром
    for epoch in tqdm(range(N_EPOCHS), desc="Фінальне тренування"):
        train_model(final_model, final_optimizer, criterion, train_loader, 1)

    final_test_loss, final_test_acc = evaluate_model(final_model, test_loader, criterion)

    print(f"\n** Фінальна точність на тестовому наборі (AdamW, LR=1e-3): {final_test_acc:.2f}%**")
    print(f"** Фінальні втрати на тестовому наборі: {final_test_loss:.4f}**")


if __name__ == "__main__":
    main_lab_work()

 Видаляємо 42 рідкісних класів (хвороб), що мають < 2 зразків.
 Успішно завантажено. Кількість класів: 682
---  Розмірність даних: 377 ознак, 682 класів ---
---  Тренування на 20 епохах ---

## 1. Дослідження впливу швидкості навчання 🧪
-> Оптимізатор: SGD, Швидкість навчання (LR): 0.0001
-> Оптимізатор: SGD, Швидкість навчання (LR): 0.001
-> Оптимізатор: SGD, Швидкість навчання (LR): 0.01
-> Оптимізатор: Adam, Швидкість навчання (LR): 0.0001
-> Оптимізатор: Adam, Швидкість навчання (LR): 0.001
-> Оптимізатор: Adam, Швидкість навчання (LR): 0.01
| Optimizer   |     LR | Train Accuracy   | Test Accuracy   |
|:------------|-------:|:-----------------|:----------------|
| SGD         | 0.0001 | 0.53%            | 0.76%           |
| SGD         | 0.001  | 3.20%            | 6.50%           |
| SGD         | 0.01   | 64.72%           | 79.08%          |
| Adam        | 0.0001 | 48.32%           | 70.23%          |
| Adam        | 0.001  | 75.81%           | 83.66%          |
| Adam        

Фінальне тренування: 100%|██████████| 20/20 [00:54<00:00,  2.73s/it]



** Фінальна точність на тестовому наборі (AdamW, LR=1e-3): 83.82%**
** Фінальні втрати на тестовому наборі: 0.4520**
